# Solving the Redistricting Problem

## Assumptions

 - We assume a 2-party system. 
 - We only use one period's worth of historical data. 
 - We also assume that all available voters voted (no one abstained) in our historical data. 
 

## Conventions

 - We call the lowest, indivisible unit a "block".
 - Blocks are partitioned into "districts". 

## Data Inputs

### Votes
The expected number of votes for each party is a key data input for our constraints. We represent this with a matrix, $\mathbf{V}$. Since we have two parties, the matrix has the following shape.

$$ \mathbf{V} \in \mathbb{R}^{|blocks| \times 2} $$

$blocks$ is a set where $|blocks|$ is the number of elements in the set. By convention, the first column of $\mathbf{V}$ will be D votes and the second column R votes. $\mathbf{V}$ could represent a single past election's results, or it could be an expected upcoming result based on an exogenous model.

### Contiguity (or Adjacency)
The second major input is a matrix specifying which blocks are contiguous with each other, meaning that they share a border. This matrix has the following shape:

$$ \mathbf{C} \in \mathbb{R}^{|blocks| \times |blocks|} $$

The elements of the matrix are defined as 

$$ c_{ij} = \left\{ \begin{array}{cc} 1 & \text{block i borders block j} \\ 0 & \text{otherwise} \end{array} \right. $$

## Objective 

Our objective uses the concept of the "efficiency gap". See [Brennan Center](https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf). The efficiency gap depends on a concept of "wasted votes". A party is using votes efficiently if 

Votes can be wasted in two ways. A vote cast for a losing candidate is "wasted", and votes cast for a winning candidate in excess of the amount needed to win are also wasted. The efficiency gap is a signed number so it can be in favor of one party or another. We arbitrarily chose to do it from the perspective of the "D" party. So a positive efficiency gap favors the R party. 

$$ \text{Efficiency Gap} = \text{D wasted votes} - \text{R wasted votes} $$

## Variables

There are several variables in the model. The key variable is a matrix where each row of the matrix represents an indivisible block (precint, county, or census area depending on the conventions of the problem). The columns represent assignment to a district. The matrix is made up of zeroes or ones, and each row must have exactly one entry equal to one, meaning that each row must be in one and only one district. $districts$ is a set where $|districts|$ is the number of elements in each set. 

$$ 
\mathbf{D} \in \{0,1\}^{|blocks| \times |districts|}. 
$$

Several other variables are necessary to set up the problem in a linear fashion. These are best explained in the context of each constraint.

## Constraints

### Each Block Is In Exactly One District

This constraint is easily expressed by saying that the sum of each row in the $D$ variable must be exactly one. 

$$ D \left[ \begin{array}{c} 1 \\ \vdots \\ 1 \end{array} \right]  = \left[ \begin{array}{c} 1 \\ \vdots \\ 1 \end{array} \right].$$

### Calculate The Number of Wasted Votes for Losing Party

To calculate wasted votes for the losing party, we first have to know which party lost and what its vote total was. We need to know this for each district that is formed as a result of the optimization. A simple `min` function is not linear, and we desire a purely linear form of the problem. This can be achieved using the "Big M" method. See [Big M](https://en.wikipedia.org/wiki/Big_M_method). Suppose we are looking at the results in just one district, with vote totals $d$ and $r$. Then define two additional variables, $wastedUnder$ and $w$. Further choose a constant $M$ that is large enough. Then define two constraints as below.

$$ \begin{align}
wastedUnder &\geq d - Mw \\
wastedUnder &\geq r - M(1-w) \\
wastedUnder &\in \mathbb{R} \\
w &\in {0,1}
\end{align} $$

If we include $wastedUnder$ in the objective function to minimize it, then the optimizer will try to reduce it. If $d$ is smaller than $r$, it will minimize it by setting $w=0$, and allowing $wastedUnder = d$.  The other constraint is nonbinding in this case. Otherwise, it will set $w=1$, making the first constraint non-bonding and setting $wastedUnder = r$. Since $w$ must be either 0 or 1, then $wastedUnder$ will be the minimum. 

### Calculate Wasted Votes for the Winning Party

Wasted votes for the winning party occur when the winning party gets more votes than is necessary to win. It is mostly simply calculate as 
$$\max (0, \text{winning votes} - \text{threshhold to win} ).$$

Because we want to avoid `max` functions, which are non-linear, we use a trick similar to the one we used for wasted votes for the losing party. Let $VotesToWin$ be the threshold to win (50% plus one). Then set up constraints as follows.

$$
\begin{align}
wastedOver & \geq 0 \\
wastedOver & \geq d - VotesToWin 
\end{align}
$$

If we include $wastedOver$ in the objective to minimize it, then it will be $0$ if the $D$ party lost and $D - VotesToWin$ otherwise. This is the value we seek.

### Enforce Equal Sizes

Equal sizes are enforced by setting a maximum district size equal to the average district size plus an additional margin. 

### Enforce Contiguity of Districts

The contiguity constraint is the most difficult part. Many different approaches have been suggested. Here are two.

 - Flow model. In this approach, the district is represented as a graph with vertices and edges. Vertices are the blocks, and two vertices share an edge if two blocks are adjoining. Then each district is modeled as a network where a fluid can flow from one block to another only if they share an edge. Designate a node as the "sink" node. If fluid can flow from any node to the sink node, then the graph is connected. 
 
 - Explicit enumeration. This is a two-phase approach. In the first phase, one generates a large (or possibly exhaustive) list of possible re-districtings that satisfy the constraints. Then, one searches among this list to find the optimal re-districting.
 
For processing considerations, we chose the first approach. 

## Optimization Model Definition

We are now able to define a general purpose function to use with different data sets and parameters.

In [1]:
using JuMP 
using GLPKMathProgInterface
using Cbc
using Gurobi
using JLD

In [2]:
function degerry(
    votes,
    contiguity_matrix, 
    number_districts,
    common_size_threshold = 0.2; 
    solver = "Cbc", 
    effgap_factor = 1,
    numeric_focus = 0
    )
    
    _V = votes
    _C = contiguity_matrix

    blocks = size(_V,1)
    districts = number_districts
    total_vote = _V * ones(2,1)

    # Do some checks
    if any(_C != _C')
        throw(ArgumentError("Contiguity matrix is not valid. It must be symmetric."))
    end
    
    if !(solver in ["Cbc","Gurobi"])
        throw(ArgumentError(string(solver, " is not a valid solver choice. Must be either Cbc or Gurobi")))
    end
    
    if solver == "Cbc"
        m = Model(solver = CbcSolver())
    else
        m = Model(solver = GurobiSolver(Presolve=0, NumericFocus=numeric_focus))
    end
    
    ## Variables

    @variable(m, 0 <= D[i=1:blocks,j=1:districts] <= 1 , Bin)
    
    ## Constraints  

    # each block can be in only one district
    @constraint(m, D * ones(districts,1) .== 1)  
    
    # Each district must have at least one block
    # @constraint(m, (D' * V) * [1;1] .>= 1)
    
    # fixed answer
#     @constraint(m, D .== [ 
#          1.0  0.0  0.0;
#          0.0  0.0  1.0;
#          0.0  0.0  1.0;
#          0.0  1.0  0.0;
#          1.0  1.0  0.0])

    # These constraints set wasted_u to the number of wasted votes for the losing party
    @variable(m, 0 <= w[i=1:districts] <= 1, Bin)
    @variable(m, wastedu[i=1:districts, j=1:2])
    M = sum(total_vote) 
    @constraint(m, wastedu .>= 0)
    @constraint(m, wastedu[:,1] .>= (D' * _V)[:,1] - M * w)
    @constraint(m, wastedu[:,2] .>= (D' * _V)[:,2] - M * (1-w))

    # These constraints set wasted_o to the number of wasted votes for the winning party
    @variable(m, wastedo[i=1:districts, j=1:2])
    @variable(m, votestowin[i=1:districts])
    @constraint(m, votestowin .== (D' * _V) * [1;1] / 2)
    @constraint(m, wastedo .>= 0)
    @constraint(m, wastedo .>= (D' * _V) - votestowin * [1 1])

    # These constraints calculate the efficiency gap
    @variable(m, effgap)
    @variable(m, abseffgap)
    @constraint(m, effgap .== ones(1,districts) * (wastedu + wastedo) * [1;-1])
    @constraint(m, abseffgap >= effgap )
    @constraint(m, abseffgap >= - effgap )

    # These constraints enforce roughly equal sizes. 
    # @variable(m, commonsize) # this approach is too slow
    fixed_common_size = sum(_V) / districts
    @constraint(m, (D' * _V) * [1;1] .<= fixed_common_size * (1+common_size_threshold))

    # These constraints enforce contiguity, but we need to allow districts with only one block
    @variable(m, 0 <= s[i=1:blocks,j=1:districts] <= 1, Bin)
    @variable(m, Y[i=1:blocks, j=1:blocks, k=1:districts])
    @constraint(m, - s .<= D)
    @constraint(m, s .<= D)
    M = 100
    for k in 1:districts
        @constraint(m, Y[:,:,k] .* _C .>= 0)
        @constraint(m, Y[:,:,k] .* (1-_C) .== 0)
        @constraint(m, Y[:,:,k] * ones(blocks,1) .<= (M-1) * D[:,k])
        for i in 1:blocks
            @constraint(m, sum(Y[i,:,k] .* _C[i,:] ) - sum(Y[:,i,k] .* _C[:,i] ) >= D[i,k] - M * s[i,k] )
        end
    end
    @constraint(m, ones(1, blocks) * s .== 1)
    
    ## Objective

    @objective(m, Min, abseffgap + sum(wastedu) + sum(wastedo) ) 
    
    @time begin
        status = solve(m)
    end
    
    res = Dict([("Model",m),
            ("Efficiency gap goal", effgap_factor),
        ("Solve Status", status), 
        ("Efficiency Gap", getvalue(abseffgap) ),
        ("Wasted Over Votes", getvalue(wastedo)),
        ("Wasted Under Votes", getvalue(wastedu)),
        ("Total Wasted Votes [D R]", ones(1,districts) * ( getvalue(wastedu) + getvalue(wastedo))),
        ("Votes By District", getvalue(D)' * _V), 
        # ("Common Size", getvalue(common_size)), 
        ("Fixed Common Size", fixed_common_size), 
        ("District Assignments", getvalue(D)), 
        ("Total Vote Share", sum(getvalue(D)' * _V,1) ), 
        ("Total Seat Share", sum( getvalue(D)' * _V .>= repmat(maximum((getvalue(D)' * _V),2),1,2), 1)  ), 
        ("Number of blocks in each district", sum(getvalue(D),1) ), 
        ("Total votes in each district", getvalue(D)' * _V * [1;1] ), 
            ("s", getvalue(s)),
            ("Y", getvalue(Y))
    ])
    
    return res
end

degerry (generic function with 2 methods)

## Example Problem

This is a simple problem used to show how the model works. 

In [3]:
C = [ 
    1 1 1 0 0;
    1 1 1 0 0;
    1 1 1 1 1;
    0 0 1 1 1;
    0 0 1 1 1]
V = [75 25; 
    60 40; 
    43*2 57*2; 
    48 52; 
    49 51]
res = degerry(V,C, 2, 0.5, solver = "Cbc")


  2.325578 seconds (122.31 k allocations: 6.693 MiB, 0.42% gc time)


Dict{String,Any} with 16 entries:
  "Wasted Under Votes"                => [97.0 0.0; 0.0 179.0]
  "Total Vote Share"                  => [318.0 282.0]
  "Efficiency gap goal"               => 1
  "Votes By District"                 => [97.0 103.0; 221.0 179.0]
  "Total votes in each district"      => [200.0, 400.0]
  "s"                                 => [0.0 1.0; 0.0 0.0; … ; 1.0 0.0; 0.0 0.…
  "Number of blocks in each district" => [2.0 3.0]
  "Model"                             => Minimization problem with:…
  "Fixed Common Size"                 => 300.0
  "Y"                                 => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;…
  "Efficiency Gap"                    => 64.0
  "Solve Status"                      => :Optimal
  "Total Seat Share"                  => [1 1]
  "Wasted Over Votes"                 => [0.0 3.0; 21.0 0.0]
  "Total Wasted Votes [D R]"          => [118.0 182.0]
  "District Assignments"              => [0.0 1.0; 0.0 1.0; … ; 1.0 0.0; 1.0 0.…

In [4]:
# print("District Assignments")
display(res["District Assignments"])
display(res["Votes By District"])
display(res["s"])
display(res["Y"])

5×2 Array{Float64,2}:
 0.0  1.0
 0.0  1.0
 0.0  1.0
 1.0  0.0
 1.0  0.0

2×2 Array{Float64,2}:
  97.0  103.0
 221.0  179.0

5×2 Array{Float64,2}:
 0.0  1.0
 0.0  0.0
 0.0  0.0
 1.0  0.0
 0.0  0.0

5×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0   0.0  0.0
 0.0  0.0  0.0  99.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [5]:
# These are the votes arranged in a matrix of size blocks x number of parties
V = [75 25; 60 40; 43 57; 48 52; 49 51]
display(V)

# This is the contiguity matrix. C_{m,n} = 1 if block m shares a border with block n, 0 otherwise.
C = [ 
    1 1 1 0 0;
    1 1 1 1 0;
    1 1 1 1 1;
    0 1 1 1 1;
    0 0 1 1 1;]
display(C)

# The example assumes the blocks are arranged as below.

# |----------|
# |A    | B  |
# |-----|    |
# |     |----|
# | C   |    |
# |     | D  |
# |     |----|
# |     | E  |
# |-----|----|


5×2 Array{Int64,2}:
 75  25
 60  40
 43  57
 48  52
 49  51

5×5 Array{Int64,2}:
 1  1  1  0  0
 1  1  1  1  0
 1  1  1  1  1
 0  1  1  1  1
 0  0  1  1  1

In [6]:
res = degerry(V,C, 3, 0.2, solver = "Gurobi")




Optimize a model with 251 rows, 125 columns and 641 nonzeros
Coefficient statistics:
  Matrix range    [0e+00, 5e+02]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 5e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 366
Variable types: 92 continuous, 33 integer (33 binary)

Root relaxation: objective 5.000000e+01, 110 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   50.00000    0    9  366.00000   50.00000  86.3%     -    0s
     0     0   50.00000    0   20  366.00000   50.00000  86.3%     -    0s
     0     0   50.00000    0   14  366.00000   50.00000  86.3%     -    0s
     0     0   50.00000    0   21  366.00000   50.00000  86.3%     -    0s
     0     0   50.00000    0   20  366.00000   50.00000  8

Dict{String,Any} with 16 entries:
  "Wasted Under Votes"                => [97.0 0.0; 0.0 40.0; 0.0 82.0]
  "Total Vote Share"                  => [275.0 225.0]
  "Efficiency gap goal"               => 1
  "Votes By District"                 => [97.0 103.0; 60.0 40.0; 118.0 82.0]
  "Total votes in each district"      => [200.0, 100.0, 200.0]
  "s"                                 => [0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 0.0…
  "Number of blocks in each district" => [2.0 1.0 2.0]
  "Model"                             => Minimization problem with:…
  "Fixed Common Size"                 => 166.667
  "Y"                                 => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;…
  "Efficiency Gap"                    => 0.0
  "Solve Status"                      => :Optimal
  "Total Seat Share"                  => [2 1]
  "Wasted Over Votes"                 => [0.0 3.0; 10.0 0.0; 18.0 0.0]
  "Total Wasted Votes [D R]"          => [125.0 125.0]
  "District Assignments"              => [0.0 0.0 1.0

In [7]:
display(res["District Assignments"])
display(res["s"])
display(res["Y"])

5×3 Array{Float64,2}:
 0.0  0.0  1.0
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 1.0  0.0  0.0

5×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  0.0  0.0

5×5×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

## Optimize Fairness With Realistic Data

First load the data.



In [8]:
# Load data
using CSV, DataFrames
WI_votes = CSV.read("data/Gerrymander County_election_data.csv")
WI_contiguity = CSV.read("data/Gerrymander County_contiguity V2.csv", rows = 73)
WI_V = convert(Array, WI_votes[:,3:4])
WI_C = convert(Array, WI_contiguity[:,2:73])
head(sort(WI_votes,cols=[:Pop],rev=true))

,County,Pop,Dem,Rep,Wasted
1,55079,940164,319819,149445,170374
2,55025,426526,205984,73065,132919
3,55133,360767,85339,145152,59813
4,55009,226778,67316,55903,11413
5,55101,188831,53408,45954,7454
6,55087,160971,50209,39563,10646


In [9]:
# Do some checks
println("Is contiguity matrix valid: ", all(WI_C == WI_C') )

println("Total votes cast: ", sum(WI_votes[:Dem]) + sum(WI_votes[:Rep])  )

sort( [ WI_votes WI_votes[:Pop]/sum(WI_votes[:Pop])], cols =[:Pop], rev = true )         

Is contiguity matrix valid: true
Total votes cast: 2939604


,County,Pop,Dem,Rep,Wasted,x1
1,55079,940164,319819,149445,170374,0.175284
2,55025,426526,205984,73065,132919,0.0795212
3,55133,360767,85339,145152,59813,0.0672612
4,55009,226778,67316,55903,11413,0.0422803
5,55101,188831,53408,45954,7454,0.0352055
6,55087,160971,50209,39563,10646,0.0300113
7,55139,156763,48167,37946,10221,0.0292268
8,55105,152307,50529,27364,23165,0.028396
9,55059,149577,45836,31609,14227,0.027887
10,55073,125834,36367,30345,6022,0.0234604


In [10]:
res3_fair = degerry(WI_V,WI_C, 3, 0.1, solver = "Gurobi", numeric_focus = 0)

Optimize a model with 32076 rows, 16004 columns and 80840 nonzeros
Coefficient statistics:
  Matrix range    [0e+00, 3e+06]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 3e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Variable types: 15569 continuous, 435 integer (435 binary)

Root relaxation: objective 4.148180e+05, 1568 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 414818.000    0   17          - 414818.000      -     -    0s
     0     0 414818.000    0   63          - 414818.000      -     -    0s
     0     0 414818.000    0   27          - 414818.000      -     -    0s
     0     0 414818.000    0   31          - 414818.000      -     -    1s
     0     0 414818.000    0   27          - 414818.000      -     -    1s
     0     

Dict{String,Any} with 16 entries:
  "Wasted Under Votes"                => [518161.0 -0.0; -0.0 365342.0; -0.0 36…
  "Total Vote Share"                  => [1.67721e6 1.26239e6]
  "Efficiency gap goal"               => 1
  "Votes By District"                 => [518145.0 536823.0; 564781.0 365342.0;…
  "Total votes in each district"      => [1.05497e6, 930123.0, 954513.0]
  "s"                                 => [0.0 0.0 0.0; 0.0 0.0 0.0; … ; 0.0 0.0…
  "Number of blocks in each district" => [24.0 21.0 27.0]
  "Model"                             => Minimization problem with:…
  "Fixed Common Size"                 => 979868.0
  "Y"                                 => [-0.0 0.0 … 0.0 -0.0; 0.0 -0.0 … 0.0 0…
  "Efficiency Gap"                    => 0.0
  "Solve Status"                      => :Optimal
  "Total Seat Share"                  => [2 1]
  "Wasted Over Votes"                 => [-0.0 9339.0; 99719.5 -0.0; 1.17029e5 …
  "Total Wasted Votes [D R]"          => [734909.0 734909.0]
  

In [12]:

CSV.write("data/Result_District_Assignments_3_fair.csv", DataFrame(res3_fair["District Assignments"]))

CSV.Sink{DateFormat{Symbol("yyyy-mm-dd"),Tuple{Base.Dates.DatePart{'y'},Base.Dates.Delim{Char,1},Base.Dates.DatePart{'m'},Base.Dates.Delim{Char,1},Base.Dates.DatePart{'d'}}},DataType}(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: dateformat"yyyy-mm-dd"
        decimal: '.'
        truestring: 'true'
        falsestring: 'false', IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1), "data/Result_District_Assignments_3_fair.csv", 9, true, String["x1", "x2", "x3"], 3, false, Val{false})

In [ ]:
res4_fair = degerry(WI_V,WI_C, 4, 0.1, solver = "Gurobi", numeric_focus = 1)

In [ ]:
CSV.write("data/Result_District_Assignments_4_fair.csv", DataFrame(res4_fair["District Assignments"]))

In [ ]:
res6_fair = degerry(WI_V,WI_C, 6, 0.1, solver = "Gurobi", numeric_focus = 0)

In [ ]:
CSV.write("data/Result_District_Assignments_6_fair.csv", DataFrame(res6_fair["District Assignments"]))

In [ ]:
res4_fair

## Create Possible Feasible Districts

In [ ]:
m_possible = res3_fair["Model"]

#@objective(m_possible, Min, 1 ) #abseffgap + sum(wastedu) + sum(wastedo) ) 

@constraint(m_possible, )

@time begin
    status = solve(m_possible)
end

random_D = getvalue(getindex(m_possible,:D))

CSV.write("data/Result_District_Assignments_3_random.csv", DataFrame(getvalue(getindex(m_possible,:D))))

In [ ]:
display( sum(random_D,1))
display( sum(res3_fair["District Assignments"], 1))

#    res = Dict([("Model",m),
#            ("Efficiency gap goal", effgap_factor),
#        ("Solve Status", status), 
println("Efficiency Gap", getvalue(getindex(m_possible,:abseffgap)) )
#        ("Wasted Over Votes", getvalue(wastedo)),
#        ("Wasted Under Votes", getvalue(wastedu)),
#        ("Total Wasted Votes [D R]", ones(1,districts) * ( getvalue(wastedu) + getvalue(wastedo))),
#        ("Votes By District", getvalue(D)' * _V), 
#        # ("Common Size", getvalue(common_size)), 
#        ("Fixed Common Size", fixed_common_size), 
#        ("District Assignments", getvalue(D)), 
#        ("Total Vote Share", sum(getvalue(D)' * _V,1) ), 
#        ("Total Seat Share", sum( getvalue(D)' * _V .>= repmat(maximum((getvalue(D)' * _V),2),1,2), 1)  ), 
#        ("Number of blocks in each district", sum(getvalue(D),1) ), 
#        ("Total votes in each district", getvalue(D)' * _V * [1;1] ), 
#            ("s", getvalue(s)),
#            ("Y", getvalue(Y))
#    ])


    

In [ ]:

_V = WI_V
_C = WI_C 

blocks = size(_V,1)
districts = number_districts
total_vote = _V * ones(2,1)

# Do some checks
if any(_C != _C')
    throw(ArgumentError("Contiguity matrix is not valid. It must be symmetric."))
end

if !(solver in ["Cbc","Gurobi"])
    throw(ArgumentError(string(solver, " is not a valid solver choice. Must be either Cbc or Gurobi")))
end

if solver == "Cbc"
    m = Model(solver = CbcSolver())
else
    m = Model(solver = GurobiSolver(Presolve=0, NumericFocus=numeric_focus))
end

## Variables

@variable(m, 0 <= D[i=1:blocks,j=1:districts] <= 1 , Bin)

## Constraints  

# each block can be in only one district
@constraint(m, D * ones(districts,1) .== 1)  

# Each district must have at least one block
# @constraint(m, (D' * V) * [1;1] .>= 1)

# fixed answer
#     @constraint(m, D .== [ 
#          1.0  0.0  0.0;
#          0.0  0.0  1.0;
#          0.0  0.0  1.0;
#          0.0  1.0  0.0;
#          1.0  1.0  0.0])

# These constraints set wasted_u to the number of wasted votes for the losing party
@variable(m, 0 <= w[i=1:districts] <= 1, Bin)
@variable(m, wastedu[i=1:districts, j=1:2])
M = sum(total_vote) 
@constraint(m, wastedu .>= 0)
@constraint(m, wastedu[:,1] .>= (D' * _V)[:,1] - M * w)
@constraint(m, wastedu[:,2] .>= (D' * _V)[:,2] - M * (1-w))

# These constraints set wasted_o to the number of wasted votes for the winning party
@variable(m, wastedo[i=1:districts, j=1:2])
@variable(m, votestowin[i=1:districts])
@constraint(m, votestowin .== (D' * _V) * [1;1] / 2)
@constraint(m, wastedo .>= 0)
@constraint(m, wastedo .>= (D' * _V) - votestowin * [1 1])

# These constraints calculate the efficiency gap
@variable(m, effgap)
@variable(m, abseffgap)
@constraint(m, effgap .== ones(1,districts) * (wastedu + wastedo) * [1;-1])
@constraint(m, abseffgap >= effgap )
@constraint(m, abseffgap >= - effgap )

# These constraints enforce roughly equal sizes. 
# @variable(m, commonsize) # this approach is too slow
fixed_common_size = sum(_V) / districts
@constraint(m, (D' * _V) * [1;1] .<= fixed_common_size * (1+common_size_threshold))

# These constraints enforce contiguity, but we need to allow districts with only one block
@variable(m, 0 <= s[i=1:blocks,j=1:districts] <= 1, Bin)
@variable(m, Y[i=1:blocks, j=1:blocks, k=1:districts])
@constraint(m, - s .<= D)
@constraint(m, s .<= D)
M = 100
for k in 1:districts
    @constraint(m, Y[:,:,k] .* _C .>= 0)
    @constraint(m, Y[:,:,k] .* (1-_C) .== 0)
    @constraint(m, Y[:,:,k] * ones(blocks,1) .<= (M-1) * D[:,k])
    for i in 1:blocks
        @constraint(m, sum(Y[i,:,k] .* _C[i,:] ) - sum(Y[:,i,k] .* _C[:,i] ) >= D[i,k] - M * s[i,k] )
    end
end
@constraint(m, ones(1, blocks) * s .== 1)

## Objective

@objective(m, Min, abseffgap + sum(wastedu) + sum(wastedo) ) 

@time begin
    status = solve(m)
end

res = Dict([("Model",m),
        ("Efficiency gap goal", effgap_factor),
    ("Solve Status", status), 
    ("Efficiency Gap", getvalue(abseffgap) ),
    ("Wasted Over Votes", getvalue(wastedo)),
    ("Wasted Under Votes", getvalue(wastedu)),
    ("Total Wasted Votes [D R]", ones(1,districts) * ( getvalue(wastedu) + getvalue(wastedo))),
    ("Votes By District", getvalue(D)' * _V), 
    # ("Common Size", getvalue(common_size)), 
    ("Fixed Common Size", fixed_common_size), 
    ("District Assignments", getvalue(D)), 
    ("Total Vote Share", sum(getvalue(D)' * _V,1) ), 
    ("Total Seat Share", sum( getvalue(D)' * _V .>= repmat(maximum((getvalue(D)' * _V),2),1,2), 1)  ), 
    ("Number of blocks in each district", sum(getvalue(D),1) ), 
    ("Total votes in each district", getvalue(D)' * _V * [1;1] ), 
        ("s", getvalue(s)),
        ("Y", getvalue(Y))
])


In [ ]:
Pkg.add("JLD")